# Статистические данные за 2020 и 2023 гг

## Изучение данных

In [1]:
# библиотеки
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
pd.options.display.float_format = '{:,.2f}'.format #отображение данных на уровне 2-х знаков
pd.set_option('display.max_columns', None) #отображение всех столбцов датафрейма

**Загружаем файлы (Excel)**

In [ ]:
data_21 = pd.read_excel('/Users/REKL1/Downloads/TLV/наработка_21_23.xlsx') #загружаем 2021-2023 год

In [ ]:
def data_info(df): # посмотрим основную информацию
    df.info()
    #display(df.head())
    print('Количество признаков:', df.shape[1], ', количество наблюдений:', df.shape[0])
    print('Количество дубликатов:', df.duplicated().sum())
    print('Количество пропусков:', df.isna().sum())

In [ ]:
data_info(data_21)

In [ ]:
#data_21 = data_21.drop_duplicates() # удаляем дубликаты
#data_22 = data_22.drop_duplicates()

In [ ]:
data_21.columns # перечень названий столбцов таблицы data

In [ ]:
data_21 = data_21.rename(columns = {'Лицевой счёт': 'contract', 'Дата договора': 'date_contract', 'Тариф': 'tariff',
       'Сумма списания': 'sum_total', 'Вид платежа(услуга)': 'service'}) 
# переименование столбцов
data_21.columns # проверяем

In [ ]:
data_21['month'] = pd.DatetimeIndex(data_21['Последний день месяца списания']).month 
#добавляем месяц наработки

In [ ]:
data_21['year'] = pd.DatetimeIndex(data_21['Последний день месяца списания']).year
# добавляем год наработки

In [ ]:
data_21 = data_21.drop(['Последний день месяца списания', 'Дата списания кастомных тарифов'], axis=1)
# удаляем лишние столбцы

In [ ]:
#data_21.head(5)

**Загружаем словари:**

In [ ]:
directory_rates = pd.read_excel('/Users/REKL1/Downloads/directory.xlsx', sheet_name='Группа тарифов')
#загружаем файл-словарь

In [ ]:
data_info(directory_rates)

In [ ]:
directory_servic = pd.read_excel('/Users/REKL1/Downloads/directory.xlsx', sheet_name='Группа услуг')
# загружаем второй словарь

In [ ]:
data_info(directory_servic)

In [ ]:
# на основании словарей классифицируем услуги и тарифы по группам
data = data_21.merge(directory_rates, on='tariff', how='left') 

In [ ]:
# на основании словарей классифицируем услуги и тарифы по группам
data = data.merge(directory_servic, on='service', how='left') 

In [ ]:
print('Пропуски в данных:', data.isna().sum()) #проверяем на пропуски

In [ ]:
#display(data.head(5))

In [ ]:
data.info() #проверяем файл на соответствие данных

Проверяем, что нет пропусков в словарях

In [ ]:
data['tariff_group'].unique()

In [ ]:
data['service_group'].unique()

In [ ]:
structure_22 = data.pivot_table(index=['tariff_group', 'year'], columns='service_group', values='sum_total', aggfunc='sum')


In [ ]:
del directory_rates, directory_servic #освобождаем память

**Проверим данные на адекватность:**

In [ ]:
data['sum_total'].describe() #изучим суммы

In [ ]:
data[data['sum_total'] <= 0]['cid'].count() #количество сумм равное или меньше 0

In [ ]:
data_0 = data[data['sum_total'] == 0].reset_index(drop=True) # нулевые значения

In [ ]:
data_0.pivot_table(
    index='month', columns='year', values='contract', aggfunc='count').plot(
    figsize=(17, 4), grid=True, style='--', title='Количество нулевых наработок');

Перечень абонентов, у которых была нулевая наработка.

In [ ]:
data_0['contract'].unique()

In [ ]:
data = data[data['sum_total'] > 0].reset_index(drop=True) #удаляем нулевые и отрицательные значения

Проверим распределение данных.

In [ ]:
data['sum_total'].hist(bins = 50, figsize = (15,4), color='#988ed5'); #распределение данных

## Статистика по наработке

In [ ]:
data.pivot_table(
    index='month', columns='year', values='sum_total', aggfunc='sum').plot(
    figsize=(17, 4), grid=True, style='--', title='Сумма наработки');

In [ ]:
video_dom = data.query(
    'tariff_group == ["Камеры", "Домофон"] and service_group == "Абонентская плата"').reset_index(drop=True)

In [ ]:
address = pd.read_excel('/Users/REKL1/Downloads/address.xlsx')
#загружаем словарь адресов

In [ ]:
# на основании словарей классифицируем услуги и тарифы по группам
video_dom = video_dom.merge(address, on='cid', how='left').reset_index(drop=True)

In [ ]:
print('Количество дубликатов:', video_dom.duplicated().sum())

In [ ]:
video_dom = video_dom.drop_duplicates()

In [ ]:
print('Пропуски в данных:', video_dom.isna().sum()) #проверяем на пропуски

In [ ]:
#video_dom

In [ ]:
video_dom = video_dom.fillna('Неизвестно')

In [ ]:
no_name = video_dom.query('address == "Неизвестно"')

In [ ]:
no_name

Если нужно выгрузить

In [ ]:
writer = pd.ExcelWriter('no_name.xlsx', engine='xlsxwriter')
no_name.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
writer.save() # Сохраним результат 

In [ ]:
data_video_dom = video_dom.pivot_table(index=['tariff_group', 'address'], columns=['year', 'month'],
                                       values='sum_total', aggfunc='sum')
#формируем данные по абонентской плате за камеры и домофоны по адресам и месяцам

In [ ]:
total_year = data.pivot_table(index='year', columns='month', values='sum_total', aggfunc='sum')
# формируем данные о наработке за год и месяц

In [ ]:
#total_year

In [ ]:
total_tarif = data.pivot_table(index=['tariff_group', 'year'], columns='month', values='sum_total', aggfunc='sum')
# формируем наработку в разрезе групп тарифов

In [ ]:
#total_tarif

In [ ]:
tarif_sum_count = data[['tariff', 'year', 'month', 'sum_total', 'cid']]
tarif_sum = tarif_sum_count.pivot_table(index=['tariff', 'year'], columns='month', values='sum_total', aggfunc=['sum'])
# наработка по всем тарифам

In [ ]:
tarif_count = tarif_sum_count[['tariff', 'year', 'month', 'cid']]
tarif_count = tarif_count.drop_duplicates().reset_index(drop=True)
tarif_count = tarif_count.pivot_table(index=['tariff', 'year'], columns='month', values='cid', aggfunc=['count'])
# количество уникальных абонентов по тарифам 

In [ ]:
#tarif_count

In [ ]:
total_service_group = data.pivot_table(index=['service_group', 'year'], columns='month', values='sum_total',
                                       aggfunc=['sum'])
#формируем данные о наработке по группам услуг

In [ ]:
sum_ab_pl = data.query('service_group == "Абонентская плата"').pivot_table(index=['tariff_group', 'year']
                                                               , columns='month', values='sum_total', aggfunc=['sum'])
# данные по абонентской плате по группам тарифов

In [ ]:
#display(sum_ab_pl)

**Для проверки (запись в excel)**

In [ ]:
total_sum_internet = data.query('tariff_group == "Интернет"').pivot_table(index=['service_group', 'year']
                                                               , columns='month', values='sum_total', aggfunc=['sum'])
# данные по наработке Интернет в разрезе групп услуг

In [ ]:
#display(total_sum_internet)

In [ ]:
data_count = data[['year', 'month', 'cid', 'tariff_group']]
data_count = data_count.drop_duplicates().reset_index(drop=True)
products = data_count.pivot_table(index=['tariff_group', 'year'], columns=['month'], values='cid', aggfunc=['count'])
# данные по количеству уникальных абонентов

In [ ]:
#products

In [ ]:
writer = pd.ExcelWriter('total_sum_1.xlsx', engine='xlsxwriter')
sum_ab_pl.to_excel(writer, 'Аб_пл_гр_услуг') # Записать ваш DataFrame в файл  
total_tarif.to_excel(writer, 'Наработка_группы_тариф')
total_year.to_excel(writer, 'Наработка в год') # Записать ваш DataFrame в файл  
total_service_group.to_excel(writer, 'Наработка_гр_услуг')
structure_22.to_excel(writer, 'Наработка_услуги_тарифы')
total_sum_internet.to_excel(writer, 'Наработка_интернет_услуги')
tarif_sum.to_excel(writer,'Наработка_тариф')
tarif_count.to_excel(writer,'Колво_тариф')
products.to_excel(writer,'Колво_абонентов')
data_video_dom.to_excel(writer,'Аб_плата_адрес_месяц')
writer.save() # Сохраним результат 

### Наработка по домофонам и камерам

In [ ]:
domofon = data.query('tariff_group == "Домофон"')

In [ ]:
#domofon.pivot_table(index=['year'], columns=['month'], values='sum_total', aggfunc=['sum'])

In [ ]:
domofon = domofon.merge(address, on='cid', how='left').reset_index(drop=True)

In [ ]:
writer = pd.ExcelWriter('domofon.xlsx', engine='xlsxwriter')
domofon.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
writer.save() # Сохраним результат 

In [ ]:
videos = data.query('tariff_group == "Камеры"')

In [ ]:
#videos.pivot_table(index=['year'], columns=['month'], values='sum_total', aggfunc=['sum'])

In [ ]:
videos = videos.merge(address, on='cid', how='left').reset_index(drop=True)

In [ ]:
writer = pd.ExcelWriter('videos.xlsx', engine='xlsxwriter')
videos.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
writer.save() # Сохраним результат 

In [ ]:
day = data.query('year == 2022 and month == [10, 11, 12]')
day = day['contract'].drop_duplicates().reset_index(drop=True)

In [ ]:
writer = pd.ExcelWriter('no_name.xlsx', engine='xlsxwriter')
day.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
writer.save() # Сохраним результат 